In [1]:
#uncomment if qiskit is not installed
#!pip install qiskit
#!pip install networkx
#!pip install pylatexenc

In [2]:
%matplotlib inline
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from scipy.optimize import minimize

from qiskit.ignis.verification import combine_counts


In [3]:
# Renewable sources: solar, on-shore and off-shore wind turbines
number_of_renewable_sources = 3
# Locations: 9 locations across the whole UK
number_of_locations = 9

# Calculate the number of qubits needed, 
# NOTE: IBMQ systems accesible have 5 qubits
nqubits = number_of_locations * number_of_renewable_sources

G = nx.complete_graph(nqubits)


# total_cost = C = 61510030000.0 # units: £
# cost_england = CE = 55382335269.6723 # ~84%
# cost_ireland = CI = 1863232599.87539 # ~3%
# cost_scotland = CS = 5375495619.29273 # ~8%
# cost_wales = CW =  3102206954.15958 # ~5%

# Coefficient adjusted for existing power units (C - total_energy_renewable_MWh*pound_per_MWh)
total_cost = C = 51212020443
cost_england = CE = 46914035269.6723
cost_scotland = CS = 1124245619.29273
cost_wales = CW = 1936176954.15958
cost_ireland = CI = 1237562599.87539


total_budget = B = 3000000000 # Defined by the user? what would be a resonable amount? UK invested in the whole sector 37bn£ since 2010, while it spends ~12bn£ each year for oil and gas

is_location_in_england  = [1,1,1,1,1,1,0,0,0]
is_location_in_scotland = [0,0,0,0,0,0,0,1,0]
is_location_in_ireland  = [0,0,0,0,0,0,1,0,0]
is_location_in_wales  =   [0,0,0,0,0,0,0,0,1]


# Calculate revenue for each energy source in all locations 
# dict{location: [energy_source]}

# specifications for solar unit per region in England
# 2952.34361657824 East Midlands
# 2913.9540091705 East of England
# 2613.61248988883 North East +  Yorkshire and the Humber + West Midlands
# 2606.83714990859 North West
# 2941.37515805011 South East + London
# 2855.45556124974 South West



location_revenue_per_unit = {
    0: [2952.34361657824, 336113.74, 153105454.50],
    1: [2913.9540091705, 336113.74, 153105454.50],
    2: [2613.61248988883, 336113.74, 153105454.50],
    3: [2606.83714990859, 336113.74, 153105454.50],
    4: [2941.37515805011, 336113.74, 153105454.50],
    5: [2855.45556124974, 336113.74, 153105454.50],
    6: [2332.31, 434451.22, 0.0],
    7: [1088.38, 1020438.50, 85798571.43],
    8: [3573.59, 768855.93, 140790000]
}

# Calculate cost for each energy source in all locations
# dict{location: [energy_source]}
location_cost_per_unit = {
    0: [300.92, 52370.72, 22696783.84],
    1: [300.92, 52370.72, 22696783.84],
    2: [300.92, 52370.72, 22696783.84],
    3: [300.92, 52370.72, 22696783.84],
    4: [300.92, 52370.72, 22696783.84],
    5: [300.92, 52370.72, 22696783.84],
    6: [273.66, 70226.14, 0.0],
    7: [116.17, 161395.70, 11626961.90],
    8: [382.92, 122708.47, 21872844.44]
}

# variable and a rough estimate, see spreadsheet for current figures.
# dict{location: [energy_source]}


current_location_unit_capacity = {
    0: [87552, 423, 3],
    1: [105329,870,10],
    2: [85035,513,9],
    3: [25810 + 114235,31 + 106,0+5],
    4: [122607,817,0],
    5: [71506 + 83526 + 47458 ,175 + 799 + 275,0],
    6: [23869, 1312, 3+3+0],
    7: [61449, 3512, 7],
    8: [55773, 708, 3]
}

# actual location capacity should be an improvement on the current one, let's say of a p% amount wrt current scenario (it can be computed or manually set)
location_unit_capacity = {
    0: [0,0,0],
    1: [0,0,0],
    2: [0,0,0],
    3: [0,0,0],
    4: [0,0,0],
    5: [0,0,0],
    6: [0,0,0],
    7: [0,0,0],
    8: [0,0,0]
}

improvement_p = 30/100

for loc in range(number_of_locations):
    for resource in range(number_of_renewable_sources):
        location_unit_capacity[loc][resource] = np.ceil(improvement_p * current_location_unit_capacity[loc][resource])

print(location_unit_capacity)

# lambda and gamma, the initial parameter guess for the cost function
lam, gam = 1.0, 1.0

# Calculate the total costs and revenues 
a_revs, b_costs = [], []

isE, isS, isW, isI = [], [], [], []

for N in range(number_of_locations):
    for R in range(number_of_renewable_sources):
        num_units = location_unit_capacity[N][R]
        a_revs.append(num_units * location_revenue_per_unit[N][R])
        b_costs.append(num_units * location_cost_per_unit[N][R])
        isE.append(is_location_in_england[N])
        isS.append(is_location_in_scotland[N])
        isI.append(is_location_in_ireland[N])
        isW.append(is_location_in_wales[N])

print(len(b_costs))
print(len(a_revs))
print(len(isE))
print(len(isS))
print(len(isI))
print(len(isW))

{0: [26266.0, 127.0, 1.0], 1: [31599.0, 261.0, 3.0], 2: [25511.0, 154.0, 3.0], 3: [42014.0, 42.0, 2.0], 4: [36783.0, 246.0, 0.0], 5: [60747.0, 375.0, 0.0], 6: [7161.0, 394.0, 2.0], 7: [18435.0, 1054.0, 3.0], 8: [16732.0, 213.0, 1.0]}
27
27
27
27
27
27


In [4]:
def cost_obj(x, G):
    """
    Given a bitstring as a solution, this function returns
    the number of edges shared between the two partitions
    of the graph.
    
    Args:
        x: str
           solution bitstring
           
        G: networkx graph
        
    Returns:
        obj: float
             Objective

    NB: 
    C = total Cost
    lam = lambda
    gam = gamma
    a_revs = a_{ij}
    b_costs = b_{ij}
    """

    obj = lam * (4*total_cost**2 + cost_england**2 + cost_scotland**2 + cost_ireland**2 + cost_wales**2)
    obj = obj + gam*total_budget**2

    for i in G.nodes():
        obj += (lam * (isE[i] + isS[i] + isW[i] + isI[i] + 4)* a_revs[i]**2 + gam * b_costs[i]**2 
                    - 2*lam*(CE*isE[i] + CS*isS[i] + CW*isW[i] + CI*isI[i] + 4*C)*a_revs[i]  - 2*gam*B*b_costs[i] ) * int(x[i])

    for i, j in G.edges():
        obj += 2 * lam * (isE[i]*isE[j] + isS[i]*isS[j] + isW[i]*isW[j] + isI[i]*isI[j] + 4) *a_revs[i] * a_revs[j] * int(x[i]) * int(x[j])
        obj += 2 * gam * b_costs[i] * b_costs[j] * int(x[i]) * int(x[j])
            
    return obj

In [5]:
def compute_expectation(counts, G):
    
    """
    Computes expectation value based on measurement results
    
    Args:
        counts: dict
                key as bitstring, val as count
           
        G: networkx graph
        
    Returns:
        avg: float
             expectation value
    """
    
    avg = 0
    sum_count = 0
    for bitstring, count in counts.items():
        
        obj = cost_obj(bitstring, G)
        avg += obj * count
        sum_count += count
        
    return avg/sum_count

In [6]:
def create_qaoa_circ(G, theta):
    
    """
    Creates a parametrized qaoa circuit
    
    Args:  
        G: networkx graph
        theta: list
               unitary parameters
                     
    Returns:
        qc: qiskit circuit
    """
    
    nqubits = len(G.nodes())
    p = len(theta)//2  # number of alternating unitaries
    qc = QuantumCircuit(nqubits)
    
    beta = theta[:p]
    gamma = theta[p:]
    
    # initial_state, make them all bell states
    for i in range(0, nqubits):
        qc.h(i)
    
    for irep in range(0, p):
        
        # problem unitary biases
        for i in list(G.nodes()):
            qc.rz(2 * gamma[irep], i)
        # problem unitary couplings
        for pair in list(G.edges()):
            qc.rzz(2 * gamma[irep], pair[0], pair[1])

        # mixer unitary
        for i in range(0, nqubits):
            qc.rx(2 * beta[irep], i)
            
    qc.measure_all()
        
    return qc

In [7]:
def get_expectation(G, p, shots=2**40):
    
    """
    Runs parametrized circuit
    
    Args:
        G: networkx graph
        p: int,
           Number of repetitions of unitaries
    """
    
    backend = Aer.get_backend('qasm_simulator')
    backend._configuration.max_shots = 2**40
    backend.shots = shots
    
    def execute_circ(theta):
        
        qc = create_qaoa_circ(G, theta)
        counts = backend.run(qc, seed_simulator=42, 
                             nshots=1024).result().get_counts()
        
        for i in range(9):
            current_counts = backend.run(qc, seed_simulator=i**2+2*i, nshots=1024).result().get_counts()
            counts = combine_counts(counts, current_counts)

        return compute_expectation(counts, G)
    
    return execute_circ

In [ ]:
expectation = get_expectation(G, p=8)

res = minimize(expectation, [1.0, 1.0], method='COBYLA')
res

In [ ]:
#!pip install pylatexenc
backend = Aer.get_backend('aer_simulator')
#backend._configuration.max_shots = 1024
backend.shots = 1024

qc_res = create_qaoa_circ(G, res.x)
# Visualise the quantum circuit
# qc_res.draw(output='mpl')

In [ ]:
# Obtain simulation results
counts = backend.run(qc_res, seed_simulator=10).result().get_counts()
strings = list(counts.keys())
probabilities = np.array([counts[string]/1024 for string in strings])

# Plot the first 5 highest scores (energy sources and corresponding locations)
num_highest = 5
perm = np.flip((np.argsort(probabilities)))
indices = perm[0:num_highest]
xs = [strings[i] for i in indices] + ["Other"]
ys = [probabilities[i] for i in indices] + [sum(probabilities[perm[num_highest:]])]


fig, ax = plt.subplots(1, 1,figsize=(7, 5))
ax.bar(xs, ys)
ax.set_xticklabels(xs, fontsize=10, rotation=-45)
ax.set_ylabel("Probability")
plt.show()

In [ ]:
backend = Aer.get_backend('aer_simulator')

counts = backend.run(qc_res, seed_simulator=42, nshots=1024).result().get_counts()

qc_res = create_qaoa_circ(G, res.x)
for i in range(3):
    current_counts = backend.run(qc_res, seed_simulator=i**2+2*i, nshots=1024).result().get_counts()
    counts = combine_counts(counts, current_counts)

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(counts)

In [ ]:
print(sum(counts.values()))


In [ ]:
print(max(counts, key=counts.get))
print(len(strings))
int_keys = [int(string,2) for string in strings]
print(int_keys)